In [1]:
from pathlib import Path
import subprocess
import os
from glob import glob
import numpy as np
import pandas as pd
from datetime import datetime
import joblib
import warnings
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.datasets import load_wine
import pickle
import seaborn as sns

In [2]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("Qwen/tokenize")

model = torch.load("Qwen/Qwen3-1.7B.pt", weights_only = False)
model.to(device)
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [3]:
import torch
print(torch.__version__)           # Should show 2.2.2+cu118
print(torch.cuda.is_available())   # Should return True

2.6.0+cu124
True


In [4]:
data_train = pd.read_json(path_or_buf="Data_MT/mt_train.jsonl", lines=True)
data_val = pd.read_json(path_or_buf="Data_MT/mt_dev.jsonl", lines=True)
data_test = pd.read_json(path_or_buf="Data_MT/mt_test.jsonl", lines=True)

In [5]:
data_train = data_train.drop('context', axis=1)
data_val = data_val.drop('context', axis=1)

In [6]:
data_train

,source,translation
0,我想咨询件事 我对别人有点敌意,ฉันอยากจะถามอะไรหน่อย ฉันรู้สึกไม่เป็นมิตรกับค...
1,我考虑细菌性或滴虫性阴道炎的可能性,ฉันกำลังคิดว่าน่าจะเกิดภาวะช่องคลอดอักเสบจากเช...
2,你好！腰痛发现有多长时间了？是腰中间疼痛还是两侧肌肉疼痛？,สวัสดี คุณมีอาการปวดหลังส่วนล่างมานานแค่ไหนแล้...
3,是不是平时勤刷牙，多刷干净点就可以减少牙结石的沉积呢？,ถ้าแปรงฟันบ่อยๆ และทำความสะอาดบ่อยๆ ก็จะสามารถ...
4,周围环境的物理化学因素也可以引起身体过敏,ปัจจัยทางกายภาพและเคมีในสภาพแวดล้อมโดยรอบ ก็สา...
...,...,...
18595,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้ฉันค่อนข้างยุ่...
18596,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้หมอค่อนข้างยุ่...
18597,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้หมอค่อนข้างยุ่...
18598,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้ฉันค่อนข้างยุ่...


In [7]:
from datasets import Dataset, DatasetDict
from transformers import (
    NllbTokenizer, 
    M2M100ForConditionalGeneration,
    Trainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import logging
from IPython.display import display, HTML
import matplotlib.pyplot as plt

class Config:
    SRC_LANG = "zho_Hans"  # Simplified Chinese
    TGT_LANG = "tha_Thai"  # Thai
    # Training hyperparameters (optimized for large dataset)
    NUM_EPOCHS = 20       # Reduced epochs for large dataset
    BATCH_SIZE = 2       # Reduced for longer sequences
    LEARNING_RATE = 1e-5 # Reduced learning rate for stability
config = Config()

In [8]:
data_val['translation']

0       ตาพร่ามัวและหมดสติเนื่องจากสมองบาดเจ็บ (ชายอาย...
1       เดือนที่แล้ว ประจำเดือนครั้งแรกมาตอนต้นเดือน ค...
2                  บางคนแพ้ ทำแล้วไม่สบาย อึดอัดและปวดบวม
3       ความเครียดเยอะ เบื่ออาหารและน้ำ จะทำให้ผอมลงแล...
4              ความรู้สึกตอนนี้คือเหมือนมีอาการประสาทหลอน
                              ...                        
2995    สวัสดี หินปูนเป็นรอยโรคที่หลงเหลืออยู่หลังจากก...
2996            พี่สาวของฉันบอกให้กิน เธอเป็นสูตินรีแพทย์
2997          วิธีการรักษาโรคประสาทอ่อน? (ชาย อายุ 31 ปี)
2998    หลังการสัมผัสเชื้อมา 3 เดือนสามารถยืนยันได้อย่...
2999    ข้อใดข้อหนึ่งก็สามารถตัดความเสี่ยงของการติดเชื...
Name: translation, Length: 3000, dtype: object

In [9]:
x_train, x_test, y_train, y_test = train_test_split(
    data_val['source'], data_val['translation'], 
    test_size=0.05,  # 20% for test set
    random_state=42,  # for reproducibility
    shuffle=True,
)

In [10]:
result = pd.concat([data_train, pd.DataFrame({"source": x_train, "translation": y_train})], axis=0)
# result = pd.concat([data_train, data_val, axis=0)


In [11]:
datasets = DatasetDict({
    'train': Dataset.from_dict({'source': result['source'], 'translation': result['translation']}),
    'validation': Dataset.from_dict({'source': x_test, 'translation': y_test}),
})

In [12]:
datasets

DatasetDict({
    train: Dataset({
        features: ['source', 'translation'],
        num_rows: 21450
    })
    validation: Dataset({
        features: ['source', 'translation'],
        num_rows: 150
    })
})

In [13]:
tokenizer.src_lang = config.SRC_LANG
tokenizer.tgt_lang = config.TGT_LANG

In [14]:
tokenizer.eos_token

'<|im_end|>'

In [15]:
print(datasets["train"].column_names)
print(tokenizer.eos_token, tokenizer.eos_token_id)

['source', 'translation']
<|im_end|> 151645


In [16]:
datasets["train"].column_names

['source', 'translation']

In [17]:
def tokenize_function(examples):
    # Prompt template - consistent format
    prompts = [f"翻译成泰语：{src}\n泰语：" for src in examples["source"]]
    targets = examples["translation"]  # Make sure this matches your column name
    
    # Create full texts (prompt + target + eos)
    full_texts = [prompt + tgt + tokenizer.eos_token for prompt, tgt in zip(prompts, targets)]

    # Tokenize full input (prompt + target)
    tokenized = tokenizer(
        full_texts,
        truncation=True,
        max_length=1140,
        padding='max_length',
        return_tensors=None,  # Don't return tensors yet
    )

    # Tokenize prompts only to get their lengths
    prompt_tokenized = tokenizer(
        prompts,
        add_special_tokens=True,  # IMPORTANT: Must match full tokenization
        truncation=True,
        max_length=1140,
        padding=False,  # Don't pad prompts
    )

    # Prepare masked labels
    labels = []
    for i, (input_ids, prompt_ids) in enumerate(zip(tokenized["input_ids"], prompt_tokenized["input_ids"])):
        label = input_ids.copy()
        prompt_length = len(prompt_ids)
        
        # Mask the prompt part (set to -100 so it's ignored in loss)
        label[:prompt_length] = [-100] * prompt_length
        
        # Handle padding tokens - also mask them
        for j, token_id in enumerate(label):
            if token_id == tokenizer.pad_token_id:
                label[j] = -100
        
        labels.append(label)

    tokenized["labels"] = labels
    return tokenized

# Apply tokenization to your dataset
tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=datasets["train"].column_names,
    desc="Tokenizing dataset"
)

Tokenizing dataset:   0%|          | 0/21450 [00:00<?, ? examples/s]

Tokenizing dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21450
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
})

In [19]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=512,  # Rank - higher = more parameters but better performance
    lora_alpha=32,  # Scaling parameter
    lora_dropout=0.1,
    target_modules=[
        "q_proj",
        "k_proj", 
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    inference_mode=False,
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

In [20]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir= './Qwen-1.7B',
    num_train_epochs=config.NUM_EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=8,
    warmup_steps = 800,
    learning_rate=config.LEARNING_RATE,
    weight_decay=0.01,
    logging_dir="./Qwen-1.7B/logs",
    logging_steps=1000,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),
    report_to="tensorboard",
    remove_unused_columns=False,                             # Saves memory; set False if using custom dataset fields
    dataloader_pin_memory=True,                             # Helpful on some systems
    dataloader_num_workers=2,                               # Speeds up data loading; adjust as needed
    optim="adamw_torch",
)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/tmp/ipykernel_27902/2915407132.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [23]:
def print_trainable_parameters(model):
    trainable = 0
    total = 0
    for param in model.parameters():
        total += param.numel()
        if param.requires_grad:
            trainable += param.numel()
    print(f"Trainable parameters: {trainable:,}")
    print(f"Total parameters:     {total:,}")
    print(f"Trainable %:          {100 * trainable / total:.2f}%")

print_trainable_parameters(model)

Trainable parameters: 557,842,432
Total parameters:     2,278,417,408
Trainable %:          24.48%


In [ ]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss


In [ ]:
modelx = trainer.model.to("cuda")

In [ ]:
modelx

In [ ]:
def translate_text(chinese_text):
    prompt = f"翻译成泰语：{chinese_text}\n泰语："
    
    # Tokenize input
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1140,
        padding="max_length",
    ).to("cude")
    
    # Move to device
    if torch.cuda.is_available():
        inputs = {k: v.to(device) for k, v in inputs.items()}
        modelx.cuda()
    
    input_length = inputs['input_ids'].shape[1]
    
    # Generate translation
    with torch.no_grad():
        generated = modelx.generate(
            **inputs,
            temperature=0.7,  # Lower for more consistent translation
            do_sample=True,
            top_p=0.8,
            num_beams=10,  # ← This parameter
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )
    
    # Extract only the generated part (skip the input prompt)
    generated_text = tokenizer.decode(
        generated[0][input_length:], 
        skip_special_tokens=True
    ).strip()
    
    return generated_text

In [ ]:
translate_text(x_test[0])

In [ ]:
import tqdm
predictions = []
for text in tqdm.tqdm(x_test):
    predictions.append(translate_text(text))

predictions

In [ ]:
ref = []
ref = y_test
ref

In [ ]:
syn = {
    '้ํา' : '้ำ',
    '่ํา' : '่ำ',
    '๊ํา' : '๊ำ',
    '๋ํา' : '๋ำ',
    'ํา' : 'ำ'
}
mem = []
for data in predictions:
    s = data
    for k,v in syn.items(): s = s.replace(k,v)
    mem.append(s)
predictions = mem

In [ ]:
from pythainlp.tokenize import word_tokenize
import sacrebleu

# Tokenize predictions and references
tokenized_preds = [" ".join(word_tokenize(pred)) for pred in predictions]
tokenized_refs = [" ".join(word_tokenize(r)) for r in ref]

# Compute BLEU with no tokenization (since we already tokenized)
score = sacrebleu.corpus_bleu(tokenized_preds, [tokenized_refs], tokenize="none")
print(f"BLEU score: {score.score:.2f}")
#30.23
#29.02

In [ ]:
with open("summit_MT/qwen3-1.7B(fine-tune-linear)(val).txt", "w", encoding="utf-8") as f:
    for item in predictions:
        f.write(f"{item}\n")

In [ ]:
answer = []
for text in tqdm.tqdm(data_test['source']):
    answer.append(translate_text(text))

In [ ]:
translate_text(data_test['source'][0])

In [ ]:
mem = []
for data in answer:
    s = data
    for k,v in syn.items(): s = s.replace(k,v)
    mem.append(s)
answer = mem

In [ ]:
with open("summit_MT/qwen3-1.7B(fine-tune-linear).txt", "w", encoding="utf-8") as f:
    for item in answer:
        f.write(f"{item}\n")